In [2]:
import pandas as pd
import numpy as np
import nbimporter
from datetime import datetime, timedelta
import glob
import os
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. LSTM

## Creating Sequences

In [1]:
def create_sequences(data, target, sequence_length):
    sequences = []
    labels = []
    
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]  # Input sequence
        label = target[i + sequence_length]  # Target value following the sequence
        sequences.append(seq)
        labels.append(label)
    
    return np.array(sequences), np.array(labels)

## Modell aufsetzen

In [3]:
def lstm_model(X_train, X_test,y_train, y_test, layers, epochs, regularization, dropout, learning_rate, early_stopping):

    model = Sequential()
   #model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(layers, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]),
                   kernel_regularizer=l2(regularization)))
    #model.add(Dropout(0.2))
    model.add(Dropout(dropout))
    #model.add(LSTM(50, return_sequences=False))
    model.add(LSTM(layers, return_sequences=False, kernel_regularizer=l2(regularization)))
    #model.add(Dropout(0.2))
    model.add(Dropout(dropout))
    #model.add(Dense(25))
    model.add(Dense(25, activation='relu', kernel_regularizer=l2(regularization))) 
    model.add(Dense(1))

    optimizer = Adam(learning_rate=learning_rate)
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Fit the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping, restore_best_weights=True)
    lstm_model = model.fit(X_train, y_train, epochs=epochs, batch_size=20, validation_data=(X_test, y_test),
                        callbacks=[early_stopping], verbose=1)
    return lstm_model, model